In [1]:
# imports
import sqlite3
import pandas as pd

In [ ]:
# create function to create SQL connection to database
def create_connection(db_file, verbose=False):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        if verbose:
            print(f'Using SQLite version: {sqlite3.version}')
            print(f'Creating Connection to {db_file}...')
        return conn
    except sqlite3.Error as e:
        print(e)

In [2]:
# create function to query a database
def select_all_query(db_file, query, verbose=False):

    conn = create_connection(db_file, verbose)
    cur = conn.cursor()
    if not query.startswith('SELECT'):
        raise ValueError('Query should begin with `SELECT`')
    
    cur.execute(query)
    rows = cur.fetchall()

    if verbose:
        for row in rows:
            print(row)

    return rows

In [7]:
# How many total Characters are there?
tot_char = select_all_query('rpg_db.sqlite3', 'SELECT COUNT(*) FROM charactercreator_character')
print('Total Characters:', tot_char[0][0])

Total Characters: 302


In [26]:
# How many [characters] of each specific subclass?

# Total clerics
sub_char_cler = select_all_query('rpg_db.sqlite3',
                                 'SELECT COUNT(*) FROM charactercreator_character as cc INNER JOIN \
                                 charactercreator_cleric as cleric on cc.character_id = cleric.character_ptr_id')
print('Total Clerics:', sub_char_cler[0][0])

Total Clerics: 75


In [27]:
# Total fighters
sub_char_fight = select_all_query('rpg_db.sqlite3',
                                 'SELECT COUNT(*) FROM charactercreator_character as cc INNER JOIN \
                                 charactercreator_fighter as fighter on cc.character_id = fighter.character_ptr_id')
print('Total Fighters:', sub_char_fight[0][0])

Total Fighters: 68


In [28]:
# Total mages (including necromancers)
sub_char_mage = select_all_query('rpg_db.sqlite3',
                                 'SELECT COUNT(*) FROM charactercreator_character as cc INNER JOIN \
                                 charactercreator_mage as mage on cc.character_id = mage.character_ptr_id')
print('Total Mages (includes Necromancers):', sub_char_mage[0][0])

Total Mages (includes Necromancers): 108


In [29]:
# Total thieves
sub_char_thief = select_all_query('rpg_db.sqlite3',
                                 'SELECT COUNT(*) FROM charactercreator_character as cc INNER JOIN \
                                 charactercreator_thief as thief on cc.character_id = thief.character_ptr_id')
print('Total Thieves:', sub_char_thief[0][0])

Total Thieves: 51


In [30]:
# How many total Items?
tot_items = select_all_query('rpg_db.sqlite3', 
                             'SELECT COUNT(*) FROM armory_item')
print('Total Items:', tot_items[0][0])

Total Items: 174


In [33]:
# How many of the Items are weapons? How many are not?

# Total weapons
tot_weapons = select_all_query('rpg_db.sqlite3', 
                             'SELECT COUNT(*) FROM armory_item as item INNER JOIN armory_weapon \
                              as weapon on item.item_id = weapon.item_ptr_id')
print('Total Weapons:', tot_weapons[0][0])

# Total non-weapons
tot_non_weapons = select_all_query('rpg_db.sqlite3', 'SELECT (SELECT COUNT(*) FROM armory_item) - \
                                  (SELECT COUNT(*) FROM armory_item as item INNER JOIN armory_weapon as weapon on item.item_id = weapon.item_ptr_id)')
print('Total Non-Weapons:', tot_non_weapons[0][0])

Total Weapons: 37
Total Non-Weapons: 137


In [62]:
# How many Items does each character have? (Return first 20 rows)

# create query to pull 20 characters with their corresponding item count from database
query = '''SELECT character_id as `Character Id`, COUNT(item_id) as `Item Count` 
FROM charactercreator_character_inventory 
GROUP BY character_id LIMIT 20;'''

# create conn variable to access database
conn = create_connection('rpg_db.sqlite3')

# create dataframe using pandas read_sql functionality
df = pd.read_sql(query, conn)
df.head()

,Character Id,Item Count
0,1,3
1,2,3
2,3,2
3,4,4
4,5,4


In [63]:
# How many Weapons does each character have? (Return first 20 rows)

# create query to pull 20 characters with their corresponding item count from database
query = '''SELECT cci.character_id as `Character Id`, COUNT(aw.item_ptr_id) as `Weapon Count`
FROM charactercreator_character_inventory as cci
INNER JOIN armory_item as ai ON cci.item_id = ai.item_id
INNER JOIN armory_weapon as aw ON ai.item_id = aw.item_ptr_id
GROUP BY cci.character_id
LIMIT 20;'''

# create conn variable to access database
conn = create_connection('rpg_db.sqlite3')

# create dataframe using pandas read_sql functionality
df = pd.read_sql(query, conn)
df.head()

,Character Id,Weapon Count
0,5,2
1,7,1
2,11,1
3,20,1
4,22,1


In [64]:
# On average, how many Items does each Character have?

# create query to find average items per character
query = '''SELECT AVG(c)
FROM(
SELECT character_id, COUNT(item_id) as c
FROM charactercreator_character_inventory
GROUP BY character_id
)
'''

# connect to db
conn = create_connection('rpg_db.sqlite3')

# create dataframe
df = pd.read_sql(query, conn)
df

,AVG(c)
0,2.97351


In [70]:
# On average, how many Weapons does each character have?

# create query to find average weapons per character
query = '''SELECT AVG(wc)
FROM (SELECT cci.character_id as `Character Id`, COUNT(aw.item_ptr_id) as wc
FROM charactercreator_character_inventory as cci
INNER JOIN armory_item as ai ON cci.item_id = ai.item_id
LEFT JOIN armory_weapon as aw ON ai.item_id = aw.item_ptr_id
GROUP BY cci.character_id)
'''

# connect to db
conn = create_connection('rpg_db.sqlite3')

# create db
df = pd.read_sql(query, conn)


df

,AVG(wc)
0,0.672185


In [71]:
# Create database file if it doesn't exist
with sqlite3.connect('buddymove_holidayiq.sqlite3') as conn:
    
    # 1. Read csv file
    df = pd.read_csv('buddymove_holidayiq.csv')
    
    # 2. DROP TABLE review IF EXISTS
    drop_query = 'DROP TABLE IF EXISTS review'
    conn.cursor().execute(drop_query)
    
    # 3. INSERT TABLE review
    df.to_sql('review', conn, index=False)
    query = 'SELECT * FROM review'
    df = pd.read_sql(query, conn)
df.head()

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


,User Id,Sports,Religious,Nature,Theatre,Shopping,Picnic
0,User 1,2,77,79,69,68,95
1,User 2,2,62,76,76,69,68
2,User 3,2,50,97,87,50,75
3,User 4,2,68,77,95,76,61
4,User 5,2,98,54,59,95,86


In [72]:
# Count how many rows you have - it should be 249!
df.shape

(249, 7)

In [75]:
# How many users who reviewed at least 100 `Nature` in the category also
# reviewed at least 100 in the `Shopping` category? - 78

# query
query = '''SELECT *
FROM review
WHERE `Nature` >= 100 AND `Shopping` >= 100
'''

# connect
conn = create_connection('buddymove_holidayiq.sqlite3')

# create dataframe, run df.describe() to obtain count
df = pd.read_sql(query, conn)
df.describe()

,Sports,Religious,Nature,Theatre,Shopping,Picnic
count,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000
mean,16.794872,121.974359,147.871795,136.705128,132.871795,137.410256
std,5.262854,23.325464,30.712897,30.230761,22.220477,32.051677
min,4.000000,79.000000,103.000000,68.000000,103.000000,79.000000
25%,12.000000,104.250000,124.000000,114.000000,116.500000,119.250000
50%,18.000000,123.000000,144.000000,131.000000,129.000000,136.000000
75%,22.000000,138.750000,163.000000,157.750000,147.000000,153.000000
max,25.000000,188.000000,247.000000,213.000000,203.000000,218.000000
